# Generate simulated data to link

In this case study, we imagine running a person linkage on a simulated 2030 Census Unedited File (CUF) --
see the main notebook for more details, including references used throughout this notebook.
This notebook creates simulated input (CUF) and reference files approximating what would be used in such a process,
emulating the methods found in publicly available descriptions of the Census Bureau's primary person linkage system, PVS.

In [1]:
import pseudopeople as psp
import os
import logging
import warnings
import os
import shutil
import datetime
from pathlib import Path
# Importing pandas for access, regardless of whether we are using it as the compute engine
import pandas
import numpy

In [2]:
%load_ext autoreload
%autoreload 1

In [3]:
%aimport vivarium_research_prl
from vivarium_research_prl import distributed_compute, utils

In [4]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
# DO NOT EDIT if this notebook is not called generate_simulated_data_small_sample.ipynb!
# This notebook is designed to be run with papermill; this cell is tagged 'parameters'
# If running with the default parameters, you can overwrite this notebook; otherwise,
# save it to another filename.
# TODO: Rename the notebook to omit 'small_sample' in the filename and omit all outputs
# from the 'canonical version'
data_to_use = 'small_sample'
output_dir = 'output'
compute_engine = 'pandas'
num_jobs = 10
cpus_per_job = 2
memory_per_job = "40GB"

In [6]:
# Parameters
data_to_use = "ri"
output_dir = "/ihme/scratch/users/zmbc/person_linkage_case_study/generate_simulated_data/"
compute_engine = "dask"
num_jobs = 10
cpus_per_job = 2
memory_per_job = "60GB"

In [7]:
print(datetime.datetime.now())

2024-02-12 15:39:24.547524


In [8]:
psp.__version__

'0.8.3.dev6+g31db93a'

In [9]:
df_ops, pd = distributed_compute.start_compute_engine(
    compute_engine,
    num_jobs=num_jobs,
    cpus_per_job=cpus_per_job,
    memory_per_job=memory_per_job,
)

<Client: 'tcp://10.158.111.17:39199' processes=10 threads=10, memory=558.80 GiB>

In [10]:
def save_file_with_ground_truth(file_name, file, ground_truth):
    # Check that file and ground truth have the same records
    assert (
        # record_id is unique
        len(file) ==
        len(df_ops.drop_duplicates(file[['record_id']]))
    )
    assert (
        len(ground_truth) ==
        len(file[['record_id']].merge(ground_truth[['record_id']], on='record_id', how='inner'))
    )

    file_path = f'{output_dir}/{data_to_use}/{file_name}.parquet'
    utils.remove_path(file_path)
    file.to_parquet(file_path)

    ground_truth_path = f'{output_dir}/{data_to_use}/{file_name}_ground_truth.parquet'
    utils.remove_path(ground_truth_path)
    ground_truth.to_parquet(ground_truth_path)

## Load pseudopeople simulated datasets

### Record ID tracking (data lineage)

We do a little bit of work here to enable tracking the "ground truth" (the simulant IDs from
pseudopeople).
We give each pseudopeople record/row a unique identifier for tracking, and then we immediately
separate the ground truth information (which we would not have if we were using real data)
from the rest of the columns (which we would have).
The ground truth is only used in the specific "ground truth" section of this notebook,
to help avoid accidentally leaking information into the case study.

Since we also combine/aggregate pseudopeople records as part of the process of generating the
simulated reference files, ground truth is a bit more complicated than you might imagine.
For example, the ground truth may tell us that a single row in a reference file is actually
a composite of several individuals, because even the deterministic linkage (by SSN) we use
here is not without error.

We handle this by tracking *all* source records used in the construction of each record in
our reference files.
This is achieved by having a table mapping composite record IDs to the "source record IDs"
(IDs of records that were directly generated by pseudopeople).
When we aggregate records, this is combined accordingly.

In [11]:
def add_unique_id_col(df, col_name='unique_id', value_prefix=''):
    if compute_engine == 'pandas' or compute_engine.startswith('modin'):
        return df.reset_index().rename(columns={'index': col_name}).assign(**{col_name: lambda df: value_prefix + df[col_name].astype(str)})
    elif compute_engine == 'dask':
        # Can use cumsum as in https://stackoverflow.com/a/60852409/ if it needs
        # to be incrementing, but we just need uniqueness

        def add_id_to_partition(df_part, partition_info=None):
            return (
                df_part
                    .assign(**{col_name: range(len(df_part))})
                    .assign(**{col_name: lambda x: (
                            value_prefix +
                            str(partition_info['number'] if partition_info is not None else 0) +
                            '_' +
                            x[col_name].astype(str)
                        ).astype('large_string[pyarrow]')}
                    )
            )

        df = df.map_partitions(add_id_to_partition)

        return df_ops.ensure_large_string_capacity(df)
    else:
        raise ValueError()

def add_unique_record_id(df, dataset_name):
    return add_unique_id_col(df, col_name='record_id', value_prefix=f'{dataset_name}_')

# Initializes a table listing the pairs between record_ids and source record_ids.
# Should only be called on "source records"; that is, records that
# come directly out of pseudopeople.
def record_id_to_single_source_record_pairs(df, source_col='record_id'):
    if source_col == 'record_id':
        # We can't have duplicate column names, so we make a new column
        # literally called 'source_col'
        df = df.assign(source_col=lambda df: df[source_col])
        source_col = 'source_col'

    return df[['record_id', source_col]].rename(columns={source_col: 'source_record_id'})

In [12]:
# Operations that aggregate records, combining the source record pairs
# between all records that are aggregated

def merge_preserving_source_records(dfs, source_record_pairings, new_record_id_prefix, *args, **kwargs):
    assert len(dfs) == len(source_record_pairings)
    for df in dfs:
        assert 'record_id' in df.columns

    on = kwargs.get('on', None)
    if on is not None:
        # If there are nulls in any of the merge columns, they can't match to anything
        dfs = [df.dropna(subset=on, how='any') for df in dfs]

    result = dfs[0]
    source_record_pairs = source_record_pairings[0]
    dfs_and_source_record_pairs_to_combine = list(zip(dfs[1:], source_record_pairings[1:]))
    for index, (df_to_merge, source_record_pairs_to_merge) in enumerate(dfs_and_source_record_pairs_to_combine):
        result = (
            result.merge(df_to_merge, *args, **kwargs)
        )
        if index == len(dfs_and_source_record_pairs_to_combine) - 1:
            # Since this is the last step, these are the record_ids that will actually be returned
            accumulate_step_record_id_prefix = new_record_id_prefix
        else:
            # A dummy intermediate -- this shouldn't be exposed to the user
            accumulate_step_record_id_prefix = f'merge_iter_{index}'

        result = add_unique_record_id(result, accumulate_step_record_id_prefix)
        source_record_pairs = df_ops.concat([
            # The pairs that were already in result
            source_record_pairs
                .rename(columns={'record_id': 'record_id_x'})
                .merge(result[['record_id', 'record_id_x']], on='record_id_x')
                .drop(columns=['record_id_x']),
            # The new ones
            source_record_pairs_to_merge
                .rename(columns={'record_id': 'record_id_y'})
                .merge(result[['record_id', 'record_id_y']], on='record_id_y')
                .drop(columns=['record_id_y']),
        ])
        result = result.drop(columns=['record_id_x', 'record_id_y'])

    return result, source_record_pairs


def dedupe_preserving_source_records(df, source_record_pairs, columns_to_dedupe, new_record_id_prefix):#, source_records_col='source_record_ids'):
    result = df_ops.drop_duplicates(df[columns_to_dedupe])
    result = add_unique_record_id(result, new_record_id_prefix)
    df_to_result_mapping = (
        df[['record_id'] + columns_to_dedupe]
            .rename(columns={'record_id': 'record_id_pre_dedupe'})
            .merge(result, on=columns_to_dedupe)
            [['record_id', 'record_id_pre_dedupe']]
    )
    result_source_record_pairs = (
        source_record_pairs
            .rename(columns={'record_id': 'record_id_pre_dedupe'})
            .merge(df_to_result_mapping, on='record_id_pre_dedupe')
            .drop(columns=['record_id_pre_dedupe'])
    )
    return result, result_source_record_pairs


def concat_preserving_source_records(dfs, source_record_pairings, new_record_id_prefix):
    dfs = [df.rename(columns={'record_id': 'record_id_pre_concat'}) for df in dfs]
    result = df_ops.concat(dfs, ignore_index=True)
    result = add_unique_record_id(result, new_record_id_prefix)

    record_id_mapping = (
        result[['record_id', 'record_id_pre_concat']]
    )
    validate_kwarg = {
        'validate': 'm:1',
    } if compute_engine != 'dask' else {} # Dask doesn't support validate

    all_source_record_pairings = df_ops.persist(df_ops.concat(source_record_pairings, ignore_index=False))
    result_source_record_pairs = df_ops.persist(
        all_source_record_pairings
            .rename(columns={'record_id': 'record_id_pre_concat'})
            .merge(record_id_mapping, on='record_id_pre_concat', **validate_kwarg)
            .drop(columns=['record_id_pre_concat'])
    )

    if compute_engine == 'dask':
        # Manual alternative to the 'validate' kwarg
        assert len(result_source_record_pairs) == len(all_source_record_pairings)

    return result.drop(columns=['record_id_pre_concat']), result_source_record_pairs

### 1040 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files -- see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [13]:
tax_years = list(range(2025, 2030))
tax_years

[2025, 2026, 2027, 2028, 2029]

In [14]:
%%time

# Combine 1040 for all years.
simulated_taxes_1040 = df_ops.concat([
    df_ops.read_parquet(
        f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_taxes_1040_{year}.parquet',
        columns=[
            'simulant_id',
            'ssn',
            'first_name',
            'middle_initial',
            'last_name',
            'mailing_address_street_number',
            'mailing_address_street_name',
            'mailing_address_unit_number',
            'mailing_address_po_box',
            'mailing_address_city',
            'mailing_address_state',
            'mailing_address_zipcode',
        ],
    )
    for year in tax_years
], ignore_index=True)
simulated_taxes_1040 = df_ops.persist(add_unique_record_id(simulated_taxes_1040, 'simulated_1040'))
simulated_taxes_1040_source_record_pairs = df_ops.persist(record_id_to_single_source_record_pairs(simulated_taxes_1040))

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    3.340000e+02
mean     3.758730e+05
std      2.211915e+05
min      3.082460e+05
25%      3.343282e+05
50%      3.412455e+05
75%      3.488688e+05
max      1.857586e+06
dtype: float64
Creating partitions of 100MB
Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    3.340000e+02
mean     3.833044e+05
std      2.368287e+05
min      3.106030e+05
25%      3.386568e+05
50%      3.466030e+05
75%      3.533012e+05
max      1.944902e+06
dtype: float64
Creating partitions of 100MB
Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    3.340000e+02
mean     3.906756e+05
std      2.536261e+05
min      3.173050e+05
25%      3.439505e+05
50%      3.508805e+05
75%      3.589048e+05
max      2.062522e+06
dtype: float64
Creating partitions of 100MB
Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    3.340000e+02
mean     3.979774e+05
std      2.689002e+05
mi

In [15]:
simulated_taxes_1040_ground_truth = simulated_taxes_1040[['record_id', 'simulant_id']]
simulated_taxes_1040 = simulated_taxes_1040.drop(columns=['simulant_id'])
simulated_taxes_1040, simulated_taxes_1040_ground_truth = df_ops.persist(simulated_taxes_1040, simulated_taxes_1040_ground_truth)
simulated_taxes_1040

,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,record_id
npartitions=10,,,,,,,,,,,,
,large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],category[unknown],large_string[pyarrow],large_string[pyarrow]
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


### W2/1099 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files --
see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [16]:
%%time

# Combine W2/1099 for all years.
simulated_w2_1099 = df_ops.concat([
    df_ops.read_parquet(
        f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_taxes_w2_and_1099_{year}.parquet',
        columns=[
            'simulant_id',
            'ssn',
            'first_name',
            'middle_initial',
            'last_name',
            'mailing_address_street_number',
            'mailing_address_street_name',
            'mailing_address_unit_number',
            'mailing_address_po_box',
            'mailing_address_city',
            'mailing_address_state',
            'mailing_address_zipcode',
        ],
    )
    for year in tax_years
], ignore_index=True)
simulated_w2_1099 = df_ops.persist(add_unique_record_id(simulated_w2_1099, 'simulated_w2_1099'))
simulated_w2_1099_source_record_pairs = df_ops.persist(record_id_to_single_source_record_pairs(simulated_w2_1099))

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    3.340000e+02
mean     5.305510e+05
std      3.342214e+05
min      4.256450e+05
25%      4.684310e+05
50%      4.775480e+05
75%      4.898670e+05
max      2.736631e+06
dtype: float64
Creating partitions of 100MB
Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    3.340000e+02
mean     5.315097e+05
std      3.498830e+05
min      4.335740e+05
25%      4.667765e+05
50%      4.768345e+05
75%      4.875395e+05
max      2.876354e+06
dtype: float64
Creating partitions of 100MB
Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    3.340000e+02
mean     5.322380e+05
std      3.655922e+05
min      4.320390e+05
25%      4.654390e+05
50%      4.754125e+05
75%      4.860588e+05
max      2.968629e+06
dtype: float64
Creating partitions of 100MB
Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    3.340000e+02
mean     5.331833e+05
std      3.793630e+05
mi

In [17]:
simulated_w2_1099_ground_truth = simulated_w2_1099[['record_id', 'simulant_id']]
simulated_w2_1099 = simulated_w2_1099.drop(columns=['simulant_id'])
simulated_w2_1099, simulated_w2_1099_ground_truth = df_ops.persist(simulated_w2_1099, simulated_w2_1099_ground_truth)
simulated_w2_1099

,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,record_id
npartitions=10,,,,,,,,,,,,
,large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],category[unknown],large_string[pyarrow],large_string[pyarrow]
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


In [18]:
simulated_taxes, simulated_taxes_source_record_pairs = concat_preserving_source_records(
    [simulated_taxes_1040, simulated_w2_1099],
    [simulated_taxes_1040_source_record_pairs, simulated_w2_1099_source_record_pairs],
    new_record_id_prefix='simulated_taxes',
)

In [19]:
simulated_taxes_1040_with_itins = simulated_taxes_1040[simulated_taxes_1040.ssn.notnull() & simulated_taxes_1040.ssn.str.startswith('9')]
simulated_taxes_1040_with_itins_source_record_pairs = simulated_taxes_1040_source_record_pairs.merge(simulated_taxes_1040_with_itins[['record_id']], on='record_id', how='inner')
simulated_taxes_1040_with_itins, simulated_taxes_1040_with_itins_source_record_pairs = df_ops.persist(simulated_taxes_1040_with_itins, simulated_taxes_1040_with_itins_source_record_pairs)

%xdel simulated_taxes_1040
%xdel simulated_taxes_1040_source_record_pairs

In [20]:
%xdel simulated_w2_1099
%xdel simulated_w2_1099_source_record_pairs

In [21]:
# "... many of the [IRS] records contain only the first four letters of the last name."
# (Brown et al. 2023, p.30, footnote 19)
# This should be updated in pseudopeople but for now we do it here.
# Note that this truncation only matters for ITIN PIKing since for SSNs that are present in SSA we use name from SSA.
PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION = 0.4 # is this a good guess at "many" in the quote above?
if compute_engine == 'dask':
    # import dask.array as np
    # rng = np.random.default_rng(seed=1234)
    # random_df = pd.from_dask_array(rng.random(len(simulated_taxes), chunks=int(np.ceil(len(simulated_taxes) / simulated_taxes.npartitions))), columns=['random'])
    # random_df.index = simulated_taxes.index
    # to_truncate = simulated_taxes.assign(random=random_df.random).random < PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION
    to_truncate, not_to_truncate = simulated_taxes.random_split([PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION, 1 - PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION], random_state=1234)
    to_truncate = to_truncate.assign(last_name=to_truncate.last_name.str[:4])
    simulated_taxes = df_ops.concat([to_truncate, not_to_truncate], ignore_index=True)
else:
    import numpy as np
    to_truncate = simulated_taxes.sample(frac=PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION, random_state=1234).index

    simulated_taxes.loc[to_truncate, 'last_name'] = simulated_taxes.loc[to_truncate, 'last_name'].str[:4]

In [22]:
simulated_taxes, simulated_taxes_source_record_pairs = df_ops.persist(simulated_taxes, simulated_taxes_source_record_pairs)

### 2030 Census Unedited File (CUF)

For now, we gloss over the data schema for addresses.
We don't know how addresses would be formatted in the CUF (and it's hard to guess, because
address is not part of the Census form), but it likely would have some of these fields
(street number, street name, etc) combined.

While PVS input files do not in general have names split into first, middle, and last,
I am guessing the CUF **would** have first name, middle initial, last name (which is how pseudopeople
generates it), because that [matches the Census questionnaire](https://www2.census.gov/programs-surveys/decennial/2020/technical-documentation/questionnaires-and-instructions/questionnaires/2020-informational-questionnaire-english_DI-Q1.pdf).

In [23]:
%%time

simulated_census_2030 = df_ops.read_parquet(f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_census_2030.parquet')
simulated_census_2030 = df_ops.persist(add_unique_record_id(simulated_census_2030, 'simulated_census_2030'))

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    3.340000e+02
mean     7.041811e+05
std      4.634839e+05
min      5.786220e+05
25%      6.192520e+05
50%      6.317130e+05
75%      6.474605e+05
max      3.862749e+06
dtype: float64
Creating partitions of 100MB
CPU times: user 2.32 s, sys: 190 ms, total: 2.51 s
Wall time: 4.29 s


In [24]:
simulated_census_2030_ground_truth = simulated_census_2030[['record_id', 'simulant_id']]
simulated_census_2030 = simulated_census_2030.drop(columns=['simulant_id'])
simulated_census_2030, simulated_census_2030_ground_truth = df_ops.persist(simulated_census_2030, simulated_census_2030_ground_truth)
simulated_census_2030

,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year,record_id
npartitions=3,,,,,,,,,,,,,,,,,,
,large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],category[unknown],large_string[pyarrow],large_string[pyarrow],category[unknown],category[unknown],category[unknown],int64,large_string[pyarrow]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [25]:
save_file_with_ground_truth('simulated_census_2030', simulated_census_2030, simulated_census_2030_ground_truth)

In [26]:
%xdel simulated_census_2030

### Simulated SSA Numident

In [27]:
%%time

simulated_ssa_numident = df_ops.read_parquet(f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_ssa_numident.parquet')

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    3.340000e+02
mean     1.539112e+08
std      4.427300e+05
min      1.515025e+08
25%      1.539650e+08
50%      1.540328e+08
75%      1.540908e+08
max      1.542853e+08
dtype: float64
Creating partitions of 5,141MB
CPU times: user 4.62 s, sys: 767 ms, total: 5.38 s
Wall time: 44.3 s


In [28]:
if data_to_use == 'ri':
    # Remove simulants who were never observed in Rhode Island -- this is unrealistic to real life!
    # We only do this to create a medium-sized set of reference files for computational testing.
    simulated_ssa_numident = (
        simulated_ssa_numident
            .merge(
                df_ops.drop_duplicates(simulated_taxes_1040_ground_truth[['simulant_id']]).assign(observed_in_taxes_1040=1),
                on='simulant_id',
                how='left',
            )
            .merge(
                df_ops.drop_duplicates(simulated_w2_1099_ground_truth[['simulant_id']]).assign(observed_in_taxes_w2_1099=1),
                on='simulant_id',
                how='left',
            )
            .merge(
                df_ops.drop_duplicates(simulated_census_2030_ground_truth[['simulant_id']]).assign(observed_in_census_2030=1),
                on='simulant_id',
                how='left',
            )
            .assign(observed_ever=lambda df: (df.observed_in_taxes_1040 == 1) | (df.observed_in_taxes_w2_1099 == 1) | (df.observed_in_census_2030 == 1))
            .pipe(lambda df: df[df.observed_ever])
            .drop(columns=["observed_in_taxes_1040", "observed_in_taxes_w2_1099", "observed_in_census_2030", "observed_ever"])
    )

In [29]:
%xdel simulated_census_2030_ground_truth

In [30]:
simulated_ssa_numident = df_ops.persist(add_unique_record_id(simulated_ssa_numident, 'simulated_ssa_numident'))
simulated_ssa_numident_source_record_pairs = df_ops.persist(record_id_to_single_source_record_pairs(simulated_ssa_numident))

In [31]:
simulated_ssa_numident_ground_truth = simulated_ssa_numident[['record_id', 'simulant_id']]
simulated_ssa_numident = simulated_ssa_numident.drop(columns=['simulant_id'])
simulated_ssa_numident, simulated_ssa_numident_ground_truth = df_ops.persist(simulated_ssa_numident, simulated_ssa_numident_ground_truth)
simulated_ssa_numident

,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date,record_id
npartitions=11,,,,,,,,,
,large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],category[unknown],category[unknown],large_string[pyarrow],large_string[pyarrow]
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [32]:
source_record_ground_truth = df_ops.persist(df_ops.concat([
    simulated_ssa_numident_ground_truth,
    simulated_taxes_1040_ground_truth,
    simulated_w2_1099_ground_truth,
]).rename(columns={'record_id': 'source_record_id'}))
source_record_ground_truth

,source_record_id,simulant_id
npartitions=31,,
,large_string[pyarrow],large_string[pyarrow]
,...,...
...,...,...
,...,...
,...,...


In [33]:
%xdel simulated_taxes_1040_ground_truth
%xdel simulated_w2_1099_ground_truth

## Create reference files

> The Census Numident – all Social Security Administration (SSA) Numident SSN records are
  edited (collapsed) to produce a Census Numident file that contains “one best-data record” for
  each SSN. All variants of name information for each SSN are retained in the Alternate Name
  Numident file, while all variants of date of birth data are retained in the Alternate DOB
  Numident. The SSN-PIK crosswalk file is used to attach a corresponding unique PIK value for
  each SSN value in the Census Numident file.

### Census Numident

Luque and Wagner, p. 4:
  
> The SSA Numident file contains all transactions ever recorded against any single SSN - with each entry
representing an addition or change (such as name changes) to the SSN record. This file is edited to
create the **Census Numident**, which contains one record for each SSN. Each SSN record in the Census
Numident contains name, DOB, sex, race, place of birth, parents’ name, citizenship status and date of death information.

and in footnote 5:

> Name edits, DOB reconciliation, and race identifiers are some of the edits conducted to produce this Numident
file. **The resulting Numident file contains the most recent name and DOB data.**

We are missing quite a few columns, since they are missing in pseudopeople's SSA Numident: race, place of birth, parents' name,
citizenship status.
However, I'm pretty sure none of these are used in linking.

In [34]:
def fill_dates(df, fill_with):
    return (
        # Replace invalid dates with nans
        pd.to_datetime(df.event_date, format='%Y%m%d', errors='coerce')
            .fillna(pandas.to_datetime('2100-01-01' if fill_with == 'latest' else '1900-01-01'))
    )

def best_data_from_columns(df, columns, best_is_latest=True):
    # We don't want to throw out events with a missing/invalid date, so we'll fill them with the value *least* likely to be chosen
    # (earlier than all values if taking the latest, later than all values if taking the earliest).
    fill_with = 'earliest' if best_is_latest else 'latest'

    result = (
        df
            # Without mutating the existing date column, get one that is actually
            # a date type and can be used for sorting.
            # Note: we actually convert this to an integer for sorting purposes, because Modin was having trouble
            # sorting by it as an actual datetime
            .assign(event_date_for_sort=lambda df: fill_dates(df, fill_with=fill_with).astype(numpy.int64) // 10 ** 9)
            .dropna(subset=columns, how='all')
            .pipe(lambda df: df_ops.drop_duplicates(df, subset='ssn', sort_col='event_date_for_sort', keep=('last' if best_is_latest else 'first')))
            [['record_id', 'ssn'] + columns]
    )

    return result, record_id_to_single_source_record_pairs(result)

best_name, best_name_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident,
    columns=['first_name', 'middle_name', 'last_name'],
)

best_date_of_birth, best_date_of_birth_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident,
    columns=['date_of_birth'],
)

best_date_of_death, best_date_of_death_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident[simulated_ssa_numident.event_type == 'death'],
    columns=['event_date'],
)
best_date_of_death = best_date_of_death.rename(columns={'event_date': 'date_of_death'})

simulated_census_numident, simulated_census_numident_source_record_pairs = df_ops.persist(merge_preserving_source_records(
    [best_name, best_date_of_birth, best_date_of_death],
    [best_name_source_record_pairs, best_date_of_birth_source_record_pairs, best_date_of_death_source_record_pairs],
    new_record_id_prefix='simulated_census_numident',
    on=['ssn'],
    how='left',
))
simulated_census_numident

,ssn,first_name,middle_name,last_name,date_of_birth,date_of_death,record_id
npartitions=11,,,,,,,
,large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow]
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


### Alternate Name Numident

Wagner and Layne, p. 9:

>  All variants of name information for each SSN are retained in the Alternate Name
Numident file...

In [35]:
simulated_alternate_name_numident, simulated_alternate_name_numident_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_ssa_numident,
    simulated_ssa_numident_source_record_pairs,
    columns_to_dedupe=['ssn', 'first_name', 'middle_name', 'last_name'],
    new_record_id_prefix='simulated_alternate_name_numident',
))
simulated_alternate_name_numident

,ssn,first_name,middle_name,last_name,record_id
npartitions=11,,,,,
,large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow]
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [36]:
df_ops.compute(df_ops.groupby_agg_small_groups(simulated_alternate_name_numident, by='ssn', agg_func=lambda x: x.size()).describe())

count    1.669062e+06
mean     1.026583e+00
std      1.608601e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.000000e+00
dtype: float64

In [37]:
# Show alternate names for the same simulated SSN
simulated_alternate_name_numident.merge(
    df_ops.groupby_agg_small_groups(simulated_alternate_name_numident[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner'
).sort_values('ssn').tail(100)

,ssn,first_name,middle_name,last_name,record_id
862,899-11-1154,Joan,Vera,Bassett,simulated_alternate_name_numident_10_119752
863,899-11-1154,Joan,<NA>,Bassett,simulated_alternate_name_numident_10_119753
928,899-15-2306,David,Grandchild,Hart,simulated_alternate_name_numident_10_119819
929,899-15-2306,David,David,Hart,simulated_alternate_name_numident_10_119820
1248,899-18-1947,Thomas,Ronald,Steadling,simulated_alternate_name_numident_10_119867
...,...,...,...,...,...
883,899-98-1297,Jaime,Timothy,Poweli,simulated_alternate_name_numident_10_121433
972,899-99-4187,Liliian,Karen,Peterman,simulated_alternate_name_numident_10_121464
973,899-99-4187,Lillian,Karen,Peterman,simulated_alternate_name_numident_10_121465
1284,899-99-7273,<NA>,Melanie,Maple,simulated_alternate_name_numident_10_121468


### Alternate DOB Numident

Wagner and Layne, p. 9:

> ... while all variants of date of birth data are retained in the Alternate DOB
Numident.

In [38]:
simulated_alternate_dob_numident, simulated_alternate_dob_numident_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_ssa_numident,
    simulated_ssa_numident_source_record_pairs,
    columns_to_dedupe=['ssn', 'date_of_birth'],
    new_record_id_prefix='simulated_alternate_dob_numident',
))
simulated_alternate_dob_numident

,ssn,date_of_birth,record_id
npartitions=11,,,
,large_string[pyarrow],large_string[pyarrow],large_string[pyarrow]
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [39]:
df_ops.compute(df_ops.groupby_agg_small_groups(simulated_alternate_dob_numident, by='ssn', agg_func=lambda x: x.size()).describe())

count    1.669062e+06
mean     1.013792e+00
std      1.166250e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.000000e+00
dtype: float64

In [40]:
simulated_alternate_dob_numident.merge(
    df_ops.groupby_agg_small_groups(simulated_alternate_dob_numident[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner'
).sort_values('ssn').tail(100)

,ssn,date_of_birth,record_id
728,898-09-1223,19330906,simulated_alternate_dob_numident_10_136303
729,898-09-1223,19894906,simulated_alternate_dob_numident_10_136304
778,898-10-3449,19510318,simulated_alternate_dob_numident_10_136321
779,898-10-3449,19510317,simulated_alternate_dob_numident_10_136322
1154,898-18-2767,19551217,simulated_alternate_dob_numident_10_136461
...,...,...,...
1061,899-99-4187,19360934,simulated_alternate_dob_numident_10_139908
738,899-99-7273,<NA>,simulated_alternate_dob_numident_10_139911
739,899-99-7273,19600221,simulated_alternate_dob_numident_10_139912
790,899-99-9400,19576807,simulated_alternate_dob_numident_10_139920


### Name/DOB Reference File

Wagner and Layne, p. 9:

> The Name and DOB Reference files are reformatted versions of the Census Numident
and includes **all possible combinations of alternate names and dates of birth, as well as
ITIN data**. All of the reference files contain SSN/ITIN and the corresponding PIK. When
an input record is linked to a reference file, the corresponding PIK is assigned. Table 1
presents the number of observations in each of the reference files.

A slightly confusing point: sometimes the Name and DOB reference files are described
as one and the same thing, and sometimes as separate.
I believe this is because **they differ only in how they are "cut" for the PVS process:**
the name reference file is cut by first and last initial,
while the DOB reference file is cut by month and day of birth.

This is described in Wagner and Layne, p.15:

> The [DOBSearch] module matches against a re-split
version of the Numident Name Reference file, splitting the data based on month and day
of birth.

Since we handle the logic of "cutting" in the linkage process itself, we generate
a single reference file here.

Note that unlike for addresses, and unlike for the pre-processing of *input* files
(as opposed to reference files), there is no explicit nickname processing/correction here.
I am fairly sure that is accurate to the PVS, which I believe assumes that nicknames
would not be present in SSA/tax records (or at least, that the real name would appear
at least once in these records).

In [41]:
simulated_name_dob_numident_records, simulated_name_dob_numident_records_source_record_pairs = df_ops.persist(merge_preserving_source_records(
    [simulated_alternate_name_numident, simulated_alternate_dob_numident],
    [simulated_alternate_name_numident_source_record_pairs, simulated_alternate_dob_numident_source_record_pairs],
    on=['ssn'],
    how='left',
    new_record_id_prefix='name_dob_numident_records',
))
simulated_name_dob_numident_records

,ssn,first_name,middle_name,last_name,date_of_birth,record_id
npartitions=11,,,,,,
,large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow]
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [42]:
# Show alternate name/DOB for the same simulated SSN
simulated_name_dob_numident_records.merge(
    df_ops.groupby_agg_small_groups(simulated_name_dob_numident_records[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner',
).sort_values('ssn').tail(100)

,ssn,first_name,middle_name,last_name,date_of_birth,record_id
2042,899-43-6600,Genevieve,Gladys,Click,19460513,name_dob_numident_records_4_26034
2043,899-43-6600,Genevieve,Gladys,Click,19460516,name_dob_numident_records_4_26035
2044,899-43-6600,<NA>,Gladys,Click,19460513,name_dob_numident_records_4_26036
2045,899-43-6600,<NA>,Gladys,Click,19460516,name_dob_numident_records_4_26037
1980,899-46-6590,Billy,Jose,Grossman,19291207,name_dob_numident_records_0_25901
...,...,...,...,...,...,...
1893,899-99-7273,<NA>,Melanie,Maple,19600221,name_dob_numident_records_6_26062
1894,899-99-7273,Lori,Melanie,Maple,<NA>,name_dob_numident_records_6_26063
1895,899-99-7273,Lori,Melanie,Maple,19600221,name_dob_numident_records_6_26064
1302,899-99-9400,Ralph,Bradley,Stendardo,19576807,name_dob_numident_records_2_18071


#### Incorporating people with ITINs

Individual Taxpayer Identification Numbers (ITINs) can be issued to people who are required to file
federal taxes but are not eligible for a Social Security Number.
The most common reason for this is being an undocumented immigrant and therefore not being authorized
to work in the United States.

People without SSNs used to be impossible to assign PIKs to.
In 2011 the NORC report stated (p. 38, footnote 19):

> NORC understands that the Census Bureau has undertaken an effort to enhance the PVS reference files with IRS
files that include Individual Taxpayer Identification Numbers (ITIN). For those people who are required to file a tax
return but do not have, and may not want an SSN—such as a non-U.S. citizen—the IRS issues the taxpayer an ITIN.
This enhancement to the PVS reference file may help to match more non-U.S citizens.

By 2014 (Wagner and Layne, p. 5):

> One of the key enhancements [made in recent years] increased the coverage of the reference files by
including records for persons with Individual Taxpayer Identification Numbers assigned
by the Internal Revenue Service (ITINs) to [along with?] the SSN-based Numident data. 

I have not found a specific description of how ITIN records are constructed in any of the
publicly-available sources.
This may be because it is straightforward, or because the tax data schema is confidential.
I assume that only IRS data is used, since no other data source that I am aware of would
report ITIN.

It is stated that the ITIN records are created directly from tax filings and not
from ITIN applications (Brown et al. p. 29, footnote 16), which is convenient
because the tax filing data is what we can simulate with
pseudopeople:

> The NUMIDENT provides the PII on the SSN-holder from the issuing agency (SSA), and that PII is used in SSN
verification. **For ITINs, the Census Bureau does not have access to the ITIN applications** to the issuing agency (IRS),
so name and DOB verification of ITINs is less reliable.

"Less reliable" is a bit confusing here, because as stated above when generating
the simulated tax data, IRS data should not contain date of birth at all.
Here, we have stayed true to this by omitting it entirely.

We assume that only 1040 filings would be used for this purpose; we wouldn't expect ITINs to
show up on employer-filed W-2/1099 forms.

In [43]:
# Analogous to the process of getting alternate names and dates of birth
# from SSA, we retain all versions of the name from taxes.
simulated_name_for_itins, simulated_name_for_itins_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_taxes_1040_with_itins.rename(columns={'middle_initial': 'middle_name'}),
    simulated_taxes_1040_with_itins_source_record_pairs,
    columns_to_dedupe=['ssn', 'first_name', 'middle_name', 'last_name'],
    new_record_id_prefix='simulated_name_for_itins',
))
simulated_name_for_itins

,ssn,first_name,middle_name,last_name,record_id
npartitions=10,,,,,
,large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow]
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [44]:
df_ops.compute(df_ops.groupby_agg_small_groups(simulated_name_for_itins, by='ssn', agg_func=lambda x: x.size()).describe())

count    59373.000000
mean         1.173783
std          0.414463
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          5.000000
dtype: float64

In [45]:
# How many records in the simulated name/DOB numident are in the ITIN range?
# With normal pseudopeople settings, this shouldn't happen, because SSN in SSA is noiseless.
# If this number is >0, we may be adding ITIN records as more alternates to existing records.
df_ops.compute(simulated_name_dob_numident_records.ssn.str.startswith('9').sum())

0

In [46]:
simulated_name_dob_reference_file, simulated_name_dob_reference_file_source_record_pairs = df_ops.persist(concat_preserving_source_records(
    [simulated_name_dob_numident_records, simulated_name_for_itins],
    [simulated_name_dob_numident_records_source_record_pairs, simulated_name_for_itins_source_record_pairs],
    new_record_id_prefix='simulated_name_dob_reference_file',
))
simulated_name_dob_reference_file

,ssn,first_name,middle_name,last_name,date_of_birth,record_id
npartitions=21,,,,,,
,large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow]
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


### GeoBase Reference File

Wagner and Layne, p. 9:

> PVS creates three other sets of reference
files containing Numident data: the **GeoBase Reference File**, the Name Reference File,
and the DOB Reference file.
The GeoBase Reference File appends addresses from administrative records attached
to Numident data, including all possible combinations of alternate names and dates of
birth for SSN. Addresses from administrative records are edited and processed through
commercial software product to clean and standardize address data. ITIN data is also
incorporated into the Geobase.

Luque and Wagner, p. 5:

> Reference files contain data from the Numident file enhanced with address
data obtained from federal AR [administrative records] files.<sup>8</sup>
The reference files, thus, contain all variants of a person’s name, DOB,
and sex, as well as current and recent addresses. These reference files are
referred to as the (PVS) Geobase reference file since addresses (a geographic component)
are appended to each person record.<sup>9</sup> It is important to note that there are
multiple Geobase reference files that are created depending on the vintage of the
incoming file to be processed through PVS.

> <sup>8</sup> Namely, data from the IRS, Department of Housing and Urban Development,
several files from the Department of Health and Human Services, and Selective Service.

> <sup>9</sup> In particular, the address data is cleaned and standardized and used
to construct a variable called GEOKEY. The GEOKEY variable is constructed as a subset
of the full address, and then is appended to the Numident data to create the
PVS Geobase Reference file.

We only have IRS data to use for addresses, and specifically only W-2/1099 data,
which is a limitation of this case study.
I can't find a concrete definition of "recent" -- as noted above, we use 5 years
of IRS data.
This may be longer than the true window, but this may end up making up for
the lack of non-IRS data sources, and get us closer to a realistic number of
alternate addresses.

Also, our address data comes out of pseudopeople already parsed into address parts
like street name, etc.
For more realism, pseudopeople should output a single string that we have to (imperfectly) parse apart.

I haven't been able to find out more about what kind of "subset" the geokey is.
It is unclear to me why geokey is "interesting" since it is just derived from the
address parts.

In [47]:
address_cols = [c for c in simulated_taxes.columns if 'mailing_address' in c]

def standardize_address_part(column):
    return (
        column
            # Remove leading or trailing whitespace
            .str.strip()
            # Turn any strings of consecutive whitespace into a single space
            .str.replace('\s+', ' ', regex=True)
            # Normalize case
            .str.upper()
            # Normalize the word street as described in the example quoted above
            # In reality, there would be many rules like this
            .str.replace('\b(STREET|STR)\b', 'ST', regex=True)
            # Make sure missingness is represented consistently
            .replace('', numpy.nan)
    )

simulated_tax_addresses = (
    simulated_taxes
        # Can only link these to the other files if they have an SSN
        .dropna(subset=['ssn'])
        [['record_id', 'ssn'] + address_cols]
        .assign(**{c: lambda df, c=c: standardize_address_part(df[c]) for c in address_cols})
        .pipe(df_ops.ensure_large_string_capacity)
)

simulated_addresses_by_ssn, simulated_addresses_by_ssn_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_tax_addresses,
    simulated_taxes_source_record_pairs,
    columns_to_dedupe=['ssn'] + address_cols,
    new_record_id_prefix='addresses_by_ssn',
))
simulated_addresses_by_ssn

,ssn,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,record_id
npartitions=40,,,,,,,,,
,large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow]
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [48]:
num_addresses = df_ops.groupby_agg_small_groups(simulated_addresses_by_ssn, by='ssn', agg_func=lambda x: x.size()).rename('size').reset_index().sort_values('size')
num_addresses

,ssn,size
npartitions=4,,
,large_string[pyarrow],int64
,...,...
,...,...
,...,...
,...,...


In [49]:
# Show some SSNs with a lot of address variation
df_ops.compute(pd.merge(num_addresses.tail(10), simulated_addresses_by_ssn, on='ssn', how='inner').sort_values('ssn'))

,ssn,size,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,record_id
0,254-37-5067,8,1746,TELL ST,<NA>,<NA>,BARRINGTON,RI,02891,addresses_by_ssn_11_62727
1,254-37-5067,8,1746,<NA>,<NA>,<NA>,BARRINGTON,RI,02874,addresses_by_ssn_11_62728
2,254-37-5067,8,1346,TELL ST,<NA>,<NA>,BARRINGTON,RI,02874,addresses_by_ssn_11_62729
3,254-37-5067,8,1746,TELL ST,<NA>,<NA>,BAERINGTON,RI,02874,addresses_by_ssn_11_62730
4,254-37-5067,8,1746,TELL ST,<NA>,<NA>,<NA>,RI,02874,addresses_by_ssn_11_62731
...,...,...,...,...,...,...,...,...,...,...
21,792-37-7935,9,1616,ROCK CREEJ VKILOA DR,<NA>,<NA>,PAWTUCKET,RI,02891,addresses_by_ssn_33_178279
22,792-37-7935,9,1616,ROCK CREEK VILLA DR,<NA>,<NA>,PAWTUKET,RI,02891,addresses_by_ssn_33_178280
23,792-37-7935,9,<NA>,ROCK CREEK VILLA DR,<NA>,<NA>,PAWTUCKET,RI,02891,addresses_by_ssn_33_178281
24,792-37-7935,9,1816,ROCK CREEK VILLA DR,<NA>,<NA>,PAWTUCKET,RI,02891,addresses_by_ssn_33_178282


In [50]:
# Rough estimate of how many rows we should have in our reference file, once we do this Cartesian product
df_ops.compute(
    len(simulated_name_dob_reference_file) *
    df_ops.groupby_agg_small_groups(simulated_addresses_by_ssn, by='ssn', agg_func=lambda x: x.size()).mean()
)

2705405.918883788

In [51]:
simulated_geobase_reference_file, simulated_geobase_reference_file_source_record_pairs = df_ops.persist(merge_preserving_source_records(
    [simulated_name_dob_reference_file, simulated_addresses_by_ssn],
    [simulated_name_dob_reference_file_source_record_pairs, simulated_addresses_by_ssn_source_record_pairs],
    on=['ssn'],
    how='left',
    new_record_id_prefix='simulated_geobase_reference_file',
))
simulated_geobase_reference_file

,ssn,first_name,middle_name,last_name,date_of_birth,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,record_id
npartitions=40,,,,,,,,,,,,,
,large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow]
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [52]:
# Actual number of rows
len(simulated_geobase_reference_file)

2902602

## Track ground truth for reference files

In [53]:
def get_simulants_of_source_records(source_record_pairs, filter_record_ids=None):
    if filter_record_ids is not None:
        source_record_pairs = source_record_pairs.pipe(filter_record_ids)

    result = (
        df_ops.drop_duplicates(source_record_pairs)
            .merge(source_record_ground_truth, on='source_record_id')
            [['record_id', 'simulant_id']]
            .pipe(df_ops.drop_duplicates)
    )
    result = result.merge(
        df_ops.groupby_agg_small_groups(result, by='record_id', agg_func=lambda x: x.simulant_id.nunique().rename('n_unique_simulants')).reset_index(),
        on='record_id',
        how='left',
    )
    return result

def get_ground_truth_for_records(source_record_pairs):
    result = df_ops.persist(get_simulants_of_source_records(source_record_pairs))

    result_nunique_describe = df_ops.compute(result['n_unique_simulants'].describe())

    if numpy.isclose(result_nunique_describe.loc['max'], 1):
        print('No collisions')
        return result

    print('Collisions:')
    display(result_nunique_describe)

    print('Simulated tax records with the most collisions:')
    most_collisions_record_id = result.sort_values('n_unique_simulants', ascending=False).head(1).record_id.iloc[0]
    most_collisions_source_record_ids = df_ops.persist(
        df_ops.drop_duplicates(source_record_pairs[source_record_pairs.record_id == most_collisions_record_id][['source_record_id']])
    )
    most_collisions_tax_filings = (
        most_collisions_source_record_ids
            .merge(df_ops.drop_duplicates(simulated_taxes_source_record_pairs[['record_id', 'source_record_id']]), on='source_record_id', how='inner')
            .merge(simulated_taxes, on='record_id', how='left')
            .merge(source_record_ground_truth, on='source_record_id', how='left')
    )
    display(df_ops.compute(most_collisions_tax_filings))

    print('Simulated SSA records with the most collisions:')
    display(
        df_ops.compute(
            most_collisions_source_record_ids
                .merge(simulated_ssa_numident.rename(columns={'record_id': 'source_record_id'}), on='source_record_id', how='inner')
                .merge(source_record_ground_truth, on='source_record_id', how='left')
        )
    )

    return result

### Simulated Census Numident

In [54]:
len(simulated_census_numident_source_record_pairs)

3464131

In [55]:
simulated_census_numident_ground_truth = get_ground_truth_for_records(simulated_census_numident_source_record_pairs)

Collisions:


count    1.770182e+06
mean     1.114250e+00
std      3.181152e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.000000e+00
Name: n_unique_simulants, dtype: float64

Simulated tax records with the most collisions:


,source_record_id,record_id,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,simulant_id


Simulated SSA records with the most collisions:


,source_record_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date,simulant_id
0,simulated_ssa_numident_4_63970,296-36-6028,Destiny,Kathryn,Stelin,19980216,Female,death,20270316,6817_361941
1,simulated_ssa_numident_7_63834,083-56-5758,Maisy,ArieJ,Proenza,29260411,Female,creation,20260411,6810_1127686


In [56]:
simulated_census_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
npartitions=33,,,
,large_string[pyarrow],large_string[pyarrow],int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


### Simulated Alternate Name Numident

In [57]:
len(simulated_alternate_name_numident_source_record_pairs)

1798888

In [58]:
simulated_alternate_name_numident_ground_truth = get_ground_truth_for_records(simulated_alternate_name_numident_source_record_pairs)

No collisions


In [59]:
simulated_alternate_name_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
npartitions=31,,,
,large_string[pyarrow],large_string[pyarrow],int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


### Alternate DOB Numident

In [60]:
len(simulated_alternate_dob_numident_source_record_pairs)

1798888

In [61]:
simulated_alternate_dob_numident_ground_truth = get_ground_truth_for_records(simulated_alternate_dob_numident_source_record_pairs)

No collisions


In [62]:
simulated_alternate_dob_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
npartitions=31,,,
,large_string[pyarrow],large_string[pyarrow],int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


### Simulated Name/DOB Reference File

In [63]:
len(simulated_name_dob_reference_file_source_record_pairs)

3876401

In [64]:
simulated_name_dob_reference_file_ground_truth = get_ground_truth_for_records(simulated_name_dob_reference_file_source_record_pairs)

No collisions


In [65]:
simulated_name_dob_reference_file_ground_truth

,record_id,simulant_id,n_unique_simulants
npartitions=32,,,
,large_string[pyarrow],large_string[pyarrow],int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


### Simulated GeoBase Reference File

In [66]:
len(simulated_geobase_reference_file_source_record_pairs)

15274707

In [67]:
simulated_geobase_reference_file_ground_truth = get_ground_truth_for_records(simulated_geobase_reference_file_source_record_pairs)

Collisions:


count    4.575453e+06
mean     1.759787e+00
std      5.225872e-01
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      2.000000e+00
max      7.000000e+00
Name: n_unique_simulants, dtype: float64

Simulated tax records with the most collisions:


,source_record_id,record_id,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,simulant_id
0,simulated_w2_1099_9_167464,simulated_taxes_19_167464,279-11-9572,Mark,D,Lodha,12569,maywood ave,<NA>,<NA>,warwick,RI,02906,7125_711463
0,simulated_w2_1099_3_164436,simulated_taxes_13_164436,279-11-9572,Mark,D,Lodha,12569,maywood ave,<NA>,<NA>,warwick,RI,02906,7125_711463
1,simulated_1040_2_550383,simulated_taxes_2_550383,279-11-9572,Robert,R,Jimenez,12569,maywood ave,<NA>,<NA>,warwick,RI,02906,7125_711453
0,simulated_w2_1099_5_164211,simulated_taxes_15_164211,279-11-9572,Margaret,G,Jimenez,12569,maywood ave,<NA>,<NA>,warwick,RI,02906,7125_711460
0,simulated_1040_8_582185,simulated_taxes_8_582185,279-11-9572,Robert,R,Jimenez,12569,maywood ave,<NA>,<NA>,warwick,RI,02906,7125_711453
0,simulated_w2_1099_7_166053,simulated_taxes_17_166053,279-11-9572,Mark,D,Lodha,12569,maywood ave,<NA>,<NA>,warwick,RI,02906,7125_711463
0,simulated_w2_1099_3_164428,simulated_taxes_13_164428,279-11-9572,Robert,R,Jime,12569,maywood ave,<NA>,<NA>,warwick,RI,02906,7125_711453
0,simulated_w2_1099_9_167462,simulated_taxes_19_167462,279-11-9572,Jacqueline,S,Jimenez,12569,maywood ave,<NA>,<NA>,warwick,RI,02906,7125_711458
0,simulated_w2_1099_7_166049,simulated_taxes_17_166049,279-11-9572,Robert,R,Jimenez,12569,maywood ave,<NA>,<NA>,warwick,RI,02906,7125_711453
0,simulated_1040_0_539737,simulated_taxes_0_539737,279-11-9572,Robert,R,Jimenez,12569,maywood ave,<NA>,<NA>,warwick,RI,02906,7125_711453


Simulated SSA records with the most collisions:


,source_record_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date,simulant_id
0,simulated_ssa_numident_5_57718,415-83-6049,Jamie,Jessica,Shandler,19940123,Female,creation,19940123,6545_389063


In [68]:
simulated_geobase_reference_file_ground_truth

,record_id,simulant_id,n_unique_simulants
npartitions=80,,,
,large_string[pyarrow],large_string[pyarrow],int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [69]:
%xdel simulated_taxes
%xdel simulated_taxes_source_record_pairs

### Get ground truth by SSN

In [70]:
simulated_ssn_simulant_pairs = df_ops.persist(
    df_ops.concat([
        simulated_census_numident[["record_id", "ssn"]].merge(simulated_census_numident_ground_truth, on="record_id"),
        simulated_alternate_name_numident[["record_id", "ssn"]].merge(simulated_alternate_name_numident_ground_truth, on="record_id"),
        simulated_alternate_dob_numident[["record_id", "ssn"]].merge(simulated_alternate_dob_numident_ground_truth, on="record_id"),
        simulated_name_dob_reference_file[["record_id", "ssn"]].merge(simulated_name_dob_reference_file_ground_truth, on="record_id"),
        simulated_geobase_reference_file[["record_id", "ssn"]].merge(simulated_geobase_reference_file_ground_truth, on="record_id"),
    ])
        [['ssn', 'simulant_id']]
        .pipe(df_ops.drop_duplicates)
)
simulated_ssn_simulant_pairs

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    2.070000e+02
mean     5.245635e+06
std      2.569957e+05
min      4.653072e+06
25%      5.160012e+06
50%      5.376750e+06
75%      5.421618e+06
max      5.488510e+06
dtype: float64
Creating partitions of 109MB


,ssn,simulant_id
npartitions=11,,
,large_string[pyarrow],large_string[pyarrow]
,...,...
...,...,...
,...,...
,...,...


In [71]:
# How many simulated SSNs have different simulant IDs
# contributing to them?
df_ops.compute((df_ops.groupby_agg_small_groups(simulated_ssn_simulant_pairs, by='ssn', agg_func=lambda x: x.simulant_id.nunique()) > 1).sum())

1712053

## Save results

In [72]:
files = {
    'simulated_census_numident': (simulated_census_numident, simulated_census_numident_ground_truth),
    'simulated_alternate_name_numident': (simulated_alternate_name_numident, simulated_alternate_name_numident_ground_truth),
    'simulated_alternate_dob_numident': (simulated_alternate_dob_numident, simulated_alternate_dob_numident_ground_truth),
    'simulated_geobase_reference_file': (simulated_geobase_reference_file, simulated_geobase_reference_file_ground_truth),
    'simulated_name_dob_reference_file': (simulated_name_dob_reference_file, simulated_name_dob_reference_file_ground_truth),
}

In [73]:
simulated_reference_files = [
    simulated_census_numident,
    simulated_alternate_name_numident,
    simulated_alternate_dob_numident,
    simulated_geobase_reference_file,
    simulated_name_dob_reference_file,
]
# TODO: Rename the ssn column to explicitly include itins, since this is confusing
simulated_all_ssns_itins_in_reference_files = df_ops.concat([df[["ssn"]] for df in simulated_reference_files], ignore_index=True)
simulated_ssn_to_pik = (
    simulated_all_ssns_itins_in_reference_files
        .pipe(df_ops.drop_duplicates)
        .pipe(add_unique_id_col, col_name='pik')
        [['ssn', 'pik']]
)
simulated_ssn_to_pik

,ssn,pik
npartitions=94,,
,large_string[pyarrow],large_string[pyarrow]
,...,...
...,...,...
,...,...
,...,...


In [74]:
simulated_pik_simulant_pairs = (
    simulated_ssn_simulant_pairs
        .merge(simulated_ssn_to_pik, on="ssn", how="inner")
        [['pik', 'simulant_id']]
)
simulated_pik_simulant_pairs

,pik,simulant_id
npartitions=94,,
,large_string[pyarrow],large_string[pyarrow]
,...,...
...,...,...
,...,...
,...,...


In [75]:
%xdel simulated_ssn_simulant_pairs

In [76]:
for file_name, (file, ground_truth) in files.items():
    file = df_ops.persist(file.merge(simulated_ssn_to_pik, on='ssn', how='left'))
    assert df_ops.compute(file.pik.notnull().all())

    save_file_with_ground_truth(file_name, file, ground_truth)

In [77]:
simulated_pik_simulant_pairs_path = f'{output_dir}/{data_to_use}/simulated_pik_simulant_pairs.parquet'
utils.remove_path(simulated_pik_simulant_pairs_path)
simulated_pik_simulant_pairs.to_parquet(simulated_pik_simulant_pairs_path)

In [78]:
print(datetime.datetime.now())

2024-02-12 16:05:32.862573
